In [51]:
from collections import defaultdict, Counter
import itertools
import functools
import numpy as np
import re
from typing import Callable, TypeVar

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

In [99]:
data1 = data(1)

def day1(match):
    sum = 0
    for line in data1:
        matched = match(line)
        first, last = matched[0], matched[-1]
        # print(first, last, sum)
        sum += int(first + last)
    return sum

print(day1(lambda x: re.findall(r'\d', x)))

def part2match(x):
    change = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
    }
    matched = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', x)
    return [change[match] if match in change else match for match in matched]

day1(part2match)
    

55816


54980

In [148]:
def format2(line):
    def tuplefy(round):
        result = [0,0,0]
        for i in round:
            result[0 if 'blue' in i else 1 if 'green' in i else 2] += int(i.split(' ')[0])
        return result

    line = line.split(': ')[-1]
    rounds = line.split('; ' )
    return np.array([tuplefy(x.split(', ')) for x in rounds])

data2 = data(2, format2)

def day2_1(start):
    result = 0
    for i, game in enumerate(data2):
        possible = np.subtract(start, game)
        if np.any(possible < 0):
            continue
        result += i+1
    return result

print(day2_1((14, 13, 12)))

def day2_2():
    result = 0
    for game in data2:
        requirements = np.amax(game, axis=0)
        result += functools.reduce(lambda x, y: x*y, requirements, 1)
    return result
day2_2()

2377


71220

In [130]:
data3 = data(3)

def day3():
    numbers = []
    for i, line in enumerate(data3):
        numbers += [(m.group(), (i, m.start())) for m in re.finditer(r'\d+', line)]

    array3 = np.pad(np.array([list(line) for line in data3]), ((1, 1), (1, 1)), constant_values='.')
    gears = defaultdict(lambda: [])

    part1 = 0
    for n, (i, j) in numbers:
        sliced = array3[i:i+3, j:j+2+len(n)]
        stringified =  ''.join(''.join(x) for x in sliced)
        if re.search(r'[^\d.]', stringified):
            part1 += int(n)
        if stars := np.where(sliced == '*'):
            for x, _ in enumerate(stars[0]):
                gears[(i+stars[0][x], j+stars[1][x])].append(int(n))

    part2 = 0
    for g in gears:
        if len(gears[g]) == 2:
            part2 += gears[g][0] * gears[g][1]

    return part1, part2

day3()

(521515, 69527306)

In [62]:
def parse4(line):
    winning, have = map(
        lambda numbers: {int(x) for x in numbers.split()},
        line.split(':')[-1].split('|')
    )
    return winning, have

data4 = data(4, parse4)

In [48]:
result = 0
for line in data4:
    matches = line[0].intersection(line[1])
    if matches:
        result += 2**(len(matches)-1)
        # print(matches, result, 2**(len(matches)-1), len(matches)-1)
print(result)

25651


In [61]:
cards = Counter()
for i, line in enumerate(data4):
    matches = line[0].intersection(line[1])
    for n in range(len(matches)):
        cards[i+1+n] += cards[i]+1
    # print(i, line, matches, cards)
print(cards.total()+len(data4))

0 ({41, 48, 17, 83, 86}, {6, 9, 48, 17, 83, 53, 86, 31}) {48, 17, 83, 86} Counter({1: 1, 2: 1, 3: 1, 4: 1})
1 ({32, 13, 16, 20, 61}, {32, 68, 17, 82, 19, 24, 61, 30}) {32, 61} Counter({2: 3, 3: 3, 1: 1, 4: 1})
2 ({1, 44, 53, 21, 59}, {1, 69, 72, 14, 16, 82, 21, 63}) {1, 21} Counter({3: 7, 4: 5, 2: 3, 1: 1})
3 ({69, 73, 41, 84, 92}, {5, 76, 51, 84, 83, 54, 58, 59}) {84} Counter({4: 13, 3: 7, 2: 3, 1: 1})
4 ({32, 83, 87, 26, 28}, {36, 70, 12, 82, 22, 88, 93, 30}) set() Counter({4: 13, 3: 7, 2: 3, 1: 1})
5 ({72, 13, 18, 56, 31}, {35, 67, 36, 74, 10, 11, 77, 23}) set() Counter({4: 13, 3: 7, 2: 3, 1: 1})
30
